In [1]:
import json
import pickle
import numpy as np
import random
import tensorflow
import time
import sys

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

In [3]:
lemmatizer = WordNetLemmatizer()

# Load the preprocessed words and classes
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Load the trained model
model = load_model('medical_chatbot_model.h5')

ignore_letters = ["?", "!", ".", ","]

# Load intents from the JSON file
with open("data.json") as file:
    intents_data = json.load(file)
    intents = intents_data["intents"]

# Function to preprocess the user's input
def preprocess_input(input_text):
    word_list = nltk.word_tokenize(input_text)
    word_list = [lemmatizer.lemmatize(word.lower()) for word in word_list if word not in ignore_letters]
    bag = [1 if word in word_list else 0 for word in words]
    return np.array(bag)

def extract_symptoms(input_text):
    word_list = nltk.word_tokenize(input_text)
    symptoms = [lemmatizer.lemmatize(word.lower()) for word in word_list if word not in ignore_letters]
    return symptoms

def predict_intent(input_text):
    processed_input = preprocess_input(input_text)
    result = model.predict(np.array([processed_input]))[0]
    max_confidence = np.max(result)
    predicted_indices = np.where(result == max_confidence)[0]
    intents_with_max_confidence = [classes[i] for i in predicted_indices]
    return intents_with_max_confidence, max_confidence

def greet_user():
    greetings = ["Hi! How can I assist you today?", "Hello! How can I help you today?"]
    return random.choice(greetings)

def chatbot_input(user_input):
    intents, confidence = predict_intent(user_input)

    # Initialize response and treatments variables
    response = "I'm sorry, I'm unable to determine the problem."
    treatments = []

    # Find the intent details from the intents list
    for intent_data in intents:
        for intent in intents_data["intents"]:
            if intent["name"] == intent_data:
                if "description" in intent:
                    response = intent["description"]
                if "treatments" in intent:
                    treatments = intent["treatments"]
                break

    return [("Chatbot:", response), (treatments), confidence]


def get_treatment(user_input):
    if "treatment" in user_input.lower() or "cure" in user_input.lower() or "treatments" in user_input.lower() or "cures" in user_input.lower() or "what are the treatments" in user_input.lower():
        return True
    else:
        return False

def chatbot_respond(user_input, treatments):
    if get_treatment(user_input):
        treatment_message = "The suggested treatments are: \n- " + ',\n- '.join(treatments)
        return [("Chatbot:", treatment_message)]
    else:
        return [("Chatbot:", "Alright! Feel free to ask if you have more questions.")]

def is_exit_message(user_input):
    exit_words = ["thank", "thanks", "goodbye", "bye", "quit"]
    return any(word in user_input.lower() for word in exit_words)

def possible_disease_enquiry(user_input):
    enquiry_words = ["possible disease","possible diseases","chances","chance","predicted disease","predicted illness","scope","predict"]
    return any(word in user_input.lower() for word in enquiry_words)

def get_possible_diseases(user_input):
    symptoms = extract_symptoms(user_input)
    possible_diseases = []

    for intent in intents_data["intents"]:
        for symptom in symptoms:
            if symptom in intent["symptoms"]:
                possible_diseases.append(intent["name"])
                break

    return possible_diseases

def get_possible_diseases_and_treatments(user_input):
    symptoms = extract_symptoms(user_input)
    possible_diseases = []
    treatments = {}

    for intent in intents_data["intents"]:
        for symptom in symptoms:
            if symptom in intent["symptoms"]:
                possible_diseases.append(intent["name"])
                treatments[intent["name"]] = intent["treatments"]
                break

    return possible_diseases, treatments

def resetting_user_input(user_input):
    if "another symptoms" in user_input.lower() or "help me with another set of symptoms" in user_input.lower() or "new set of symptoms" in user_input.lower() or "fresh start" in user_input.lower() or "from beginning" in user_input.lower():
        return True
    else:
        return False
    

def typing_effect(message):
    for char in message:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(0.05)
    print("")


In [6]:
#Continue your code on what are the symptoms of a given disease? and what are its treatments?
typing_effect(greet_user())
symptoms_input = ""
possible_diseases = []
possible_treatments = {}
    
while True:
    user_input = input("User: ")
    
    if resetting_user_input(user_input):
        symptoms_input = ""
        possible_diseases = []
        possible_treatments = {}
        typing_effect("Sure, how can I help you")
        continue
       
    if is_exit_message(user_input):
        typing_effect("Chatbot: You're welcome! Take care!")
        break

    symptoms_input += " " + user_input
    response, treatments, confidence = chatbot_input(symptoms_input)

    if confidence < 0.45:
        if possible_disease_enquiry(user_input):
            possible_diseases, possible_treatments = get_possible_diseases_and_treatments(symptoms_input)
            if len(possible_diseases)>0:
                typing_effect(f"Chatbot: The possible diseases based on your symptoms are: {', '.join(possible_diseases)}")
            else:
                typing_effect("No other predictions as per my knowledge")
            continue
            
        if get_treatment(user_input):
            if len(possible_diseases) > 0:
                for disease in possible_diseases:
                    typing_effect(f"Chatbot: The treatments for {disease} are: {', '.join(possible_treatments[disease])}")
            else:
                typing_effect("Chatbot: You haven't asked for possible diseases yet. Please ask for possible diseases first.")
            continue

        common_symptoms_response = ["Your symptoms are quite common, could you please provide more details or any other symptom?", 
                                   "That is a very common symptom, provide me with another symptom you are facing","That's a quite common symptom for many diseases, provide me with any another symptom you are having?"]
        sol = random.choice(common_symptoms_response)
        typing_effect(f"Chatbot: {sol}")
        typing_effect(f"Confidence: {confidence}")
        continue

    for item in response:
        print(*item)

    typing_effect(f"Confidence: {confidence}")
    
    user_input = input("User: ")
    
    if get_treatment(user_input):
        typing_effect(f"Chatbot: The treatments are: {treatments}")
        continue
    
    if resetting_user_input(user_input):
        symptoms_input = ""
        possible_diseases = []
        possible_treatments = {}
        typing_effect("Sure, how can I help you")
        continue      
        
    if is_exit_message(user_input):
        typing_effect("Chatbot: You're welcome! Take care!")
        break


Hello! How can I help you today?
User: I have urine pink
1/1 [==============================] - 0s 18ms/step
Chatbot: That's a quite common symptom for many diseases, provide me with any another symptom you are having?
Confidence: 0.010861008428037167
User: thanks
Chatbot: You're welcome! Take care!


In [10]:
    #What are the treatments for a predicted disease